In [20]:

import torch
import numpy as np
import matplotlib.pyplot as plt
import os
from einops import rearrange
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch import nn
from typing import Any
import glob
import re
from datetime import datetime
import random
from collections import defaultdict
import numpy as np
import onnxruntime

import cv2

random.seed(42)

In [21]:
from models.allcnn2d import AllCNN2D, AllCNN2D_Prod
from drawing.interactive import draw_image

# Global

In [22]:
DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"

# Paths

In [23]:
file_path: str = os.path.abspath(".")
root_path: str = os.path.join(file_path, os.pardir, os.pardir)
checkpoint_path: str = os.path.join(
    root_path, 
    "checkpoints", 
    "GeckoFull_epoch1980_trainacc0.90605_valacc0.98089_Tloss0.78068_Vloss0.20101_lr6.320194197753456e-11.pkl"
)


# Load Model

In [24]:
label_map: list[str] = ['(', ')', '+', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'λ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '×', '÷']

model: AllCNN2D_Prod = AllCNN2D_Prod(
    labels_map=label_map,
    **{
        "conv_features": (1, 16, 32, 32, 32, 32),
        "fully_connected_features": (64, 44),
        "expected_input_size": (64, 64),
        "device": "cpu",
        "conv_dropout": 0.0,
        "verbose": True,
        "name_prefix": "GeckoFinal",
        #"checkpoint_path": checkpoint_path
    }

).eval()

Layer (type:depth-idx)                   Output Shape              Param #
AllCNN2D_Prod                            [1, 44]                   --
├─ModuleList: 1-1                        --                        --
│    └─Sequential: 2-1                   [1, 16, 32, 32]           --
│    │    └─Conv2d: 3-1                  [1, 16, 64, 64]           160
│    │    └─Dropout2d: 3-2               [1, 16, 64, 64]           --
│    │    └─BatchNorm2d: 3-3             [1, 16, 64, 64]           32
│    │    └─LeakyReLU: 3-4               [1, 16, 64, 64]           --
│    │    └─Conv2d: 3-5                  [1, 16, 32, 32]           2,320
│    │    └─Dropout2d: 3-6               [1, 16, 32, 32]           --
│    │    └─BatchNorm2d: 3-7             [1, 16, 32, 32]           32
│    │    └─LeakyReLU: 3-8               [1, 16, 32, 32]           --
│    └─Sequential: 2-2                   [1, 32, 16, 16]           --
│    │    └─Conv2d: 3-9                  [1, 32, 32, 32]           4,640
│    │  

In [25]:
[(i, char, ord(char)) for i, char in enumerate(label_map)]

[(0, '(', 40),
 (1, ')', 41),
 (2, '+', 43),
 (3, '-', 45),
 (4, '.', 46),
 (5, '0', 48),
 (6, '1', 49),
 (7, '2', 50),
 (8, '3', 51),
 (9, '4', 52),
 (10, '5', 53),
 (11, '6', 54),
 (12, '7', 55),
 (13, '8', 56),
 (14, '9', 57),
 (15, 'λ', 955),
 (16, 'a', 97),
 (17, 'b', 98),
 (18, 'c', 99),
 (19, 'd', 100),
 (20, 'e', 101),
 (21, 'f', 102),
 (22, 'g', 103),
 (23, 'h', 104),
 (24, 'i', 105),
 (25, 'j', 106),
 (26, 'k', 107),
 (27, 'l', 108),
 (28, 'm', 109),
 (29, 'n', 110),
 (30, 'o', 111),
 (31, 'p', 112),
 (32, 'q', 113),
 (33, 'r', 114),
 (34, 's', 115),
 (35, 't', 116),
 (36, 'u', 117),
 (37, 'v', 118),
 (38, 'w', 119),
 (39, 'x', 120),
 (40, 'y', 121),
 (41, 'z', 122),
 (42, '×', 215),
 (43, '÷', 247)]

In [26]:

# Create a dummy input tensor
dummy_input = torch.randn(1, 1, 64, 64)  # Example input

# Define the ONNX file path
onnx_file_path = "model.onnx"

# Export the model
torch.onnx.export(
    model,
    dummy_input,
    onnx_file_path,
    input_names=["input"],  # Name of the input layer
    output_names=["logits", "softmax", "softmax_ordered"],  # Names of the output layers
    dynamic_axes={"input": {0: "batch_size"},  # Variable batch size
                  "logits": {0: "batch_size"},
                  "softmax": {0: "batch_size"},
                  "softmax_ordered": {0: "batch_size"}},
    opset_version=11  # Specify the ONNX opset version
)

print(f"Model saved to {onnx_file_path}")

Model saved to model.onnx


# Load Onnx

In [27]:
import onnxruntime
import numpy as np

# Load the ONNX model
model_path = "model.onnx"
session = onnxruntime.InferenceSession(model_path)

input_image = np.random.random(
    (
        1,  # batch: stack as many images as you like here
        1,  # channels: needs to be 1 (grayscale), pixels are 1.0 or 0.0
        64, # height: fixed to 64 pixels for now
        64  # width: fixed to 64 pixels for now
    )
).astype(np.float32)

# Run inference
inputs: list[onnxruntime.NodeArg] = session.get_inputs()
outputs: list[onnxruntime.NodeArg] = session.get_outputs()

input_name: list[str] = inputs[0].name
output_names: list[str] = [out.name for out in outputs]


In [34]:

softmax: np.ndarray
softmax_ordered: np.ndarray
logits: np.ndarray

logits, softmax, softmax_ordered = session.run(
    output_names, 
    {input_name: input_image}
)

# logits.shape is shape (batch, character) for all character labels
# softmax.shape is shape (batch, character) for all character labels
# softmax_ordered is shape (batch, character, [label index, label prob, unicode character value])

# character dim is 44 (there are 44 character labels)
# label index is from 0 to 44 (corresponding to each ordered label index)
# label prob is a softmaxed probability for this label prediction
# unicode character value is the unicode character for this prediction



In [38]:
logits.shape, softmax.shape, softmax_ordered.shape

((1, 44), (1, 44), (1, 44, 3))

In [ ]:


top_character_probs: list[list[float]] = softmax_ordered[:, :, 1].tolist()

top_characters: list[list[str]] = [
    [
        chr(int(softmax_ordered[batch_i, i, 2])) 
        for i in range(softmax_ordered.shape[1])
    ] for batch_i in 
    range(softmax_ordered.shape[0])
]
